In [6]:
from pydub import AudioSegment

def read_audio_properties(file_path):
    audio_file = AudioSegment.from_mp3(file_path)
    return audio_file.channels, audio_file.frame_rate, int(audio_file.duration_seconds)

In [13]:
import pandas as pd
import os
from config import *
from time import time

In [16]:
audio_properties = pd.DataFrame(columns=['file_name', 'channels', 'frame_rate', 'duration'])

start = time()
for member, label in MEMBER_TO_LABEL.items():
    print(member, end=' ')
    folder = f'data/{label}'
    file_list = os.listdir(folder)
    for file in file_list:
        channels, frame_rate, duration = read_audio_properties(f'{folder}/{file}')
        audio_properties = audio_properties.append({
            'file_name': f'{label}/{file.split(".")[0]}',
            'channels': channels,
            'frame_rate': frame_rate,
            'duration': duration
        }, ignore_index=True)
        
print(f'\nElapsed time: {time() - start:.2f}s')

Jin Suga J-Hope RM Jimin V Jungkook 

In [17]:
audio_properties['channels'].value_counts(normalize=True)

2    1.0
Name: channels, dtype: float64

In [18]:
audio_properties['frame_rate'].value_counts(normalize=True)

44100    1.0
Name: frame_rate, dtype: float64

In [19]:
audio_properties['duration'].value_counts(normalize=True)

5    0.764968
3    0.101274
4    0.084076
2    0.049682
Name: duration, dtype: float64